In [1]:
import pandas as pd
import numpy as np

In [30]:
app_df = pd.read_csv('application_data.csv', usecols=['filing_date', 'application_number', 
                                                      'application_invention_type', 'uspc_class', 'uspc_subclass'])

/Users/trevorbernard/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
correspondence_df = pd.read_csv('correspondence_address.csv', usecols=['application_number', 'correspondence_name', 
                                                                       'customer_number'])

In [31]:
df = app_df.merge(correspondence_df, how='left', left_on='application_number', right_on='application_number')

In [32]:
df = df.dropna(subset=['filing_date', 'correspondence_name'])
df.to_csv('merged_patent_data.csv')

## Compute the number of patents filed each year

In [53]:
# Make a column for the year
df['Year'] = df['filing_date'].apply(lambda x: int(x[:4]))

# Drop duplicate patents
df.drop_duplicates(subset=['application_number'], inplace=True)

# Group by year and filer and sum the number of patents they filed in that year
summed = df.groupby(['correspondence_name', 'Year']).count()['application_number']
summed.to_csv('summed_data.csv')

summed = pd.read_csv('summed_data.csv')
summed.rename(columns={'application_number':'patents filed'})
summed.to_csv('summed_data.csv')

## Put in the industries to our dataset

Idea:
1. Make a dictionary that maps patent code => industry
2. Apply the dictionary to the patent codes in the merged patent data to get an industry classification for each patent
3. For each company, look for the patent code which is used most frequently, and store that as the industry

Industries:
1.  Communication Services
2.  Automobiles (technically in Consumer Discretionary sector)
3.  Internet & Direct Marketing (technically in Consumer Discretionary sector)
4.  Consumer Discretionary
5.  Consumer Staples
6.  Energy
7.  Financials
8.  Healthcare
9.  Industrials
10. Information Technology
11. Materials
12. Real Estate
13. Utilities

Ones I'm unsure of: 27, 33, 47, 109, 119, 135, 168, 169, 431, 460, 504, 800. 

#### Note: dropped D99 which was misc. since we don't actually know what industry to assign that to. Could do industrials since it seems to be the largest, but we'd just be likely to get bias there.

# Note: need to recast the industry first. Make the floats into ints and then make everything a string with 3 characters

In [79]:
industries = {'002':'Consumer Discretionary', '004':'Consumer Staples', 
              '005':'Consumer Staples', '007':'Consumer Staples', 
              '008':'Materials', '012':'Consumer Discretionary', 
              '014':'Industrials', '015':'Consumer Staples', 
              '016':'Industrials', '019':'Consumer Discretionary', 
              '023':'Materials', '024':'Consumer Discretionary',
              '026':'Consumer Discretionary', 
              '027':'Consumer Discretionary',
              '028':'Consumer Discretionary', '029':'Materials', 
              '030':'Consumer Staples', '033':'Consumer Discretionary', 
              '034':'Materials', '036':'Consumer Discretionary', 
              '037':'Industrials', '038':'Consumer Discretionary',  
              '040':'Consumer Discretionary', '042':'Industrials', 
              '043':'Consumer Discretionary', '044':'Energy',
              '047':'Health Care', '048':'Energy', 
              '049':'Automobiles', '051':'Industrials', 
              '052':'Consumer Discretionary', '053':'Materials', 
              '054':'Consumer Discretionary', '055':'Energy',
              '056':'Industrials', '057':'Consumer Discretionary',
              '059':'Industrials', '060':'Industrials', 
              '062':'Consumer Staples',
              '063':'Consumer Discretionary', '065':'Materials',
              '066':'Consumer Discretionary', 
              '068':'Consumer Discretionary',
              '069':'Materials', '070':'Consumer Staples', 
              '071':'Materials', '072':'Materials', 
              '073':'Consumer Staples', '074':'Industrials',
              '075':'Materials', '076':'Materials', 
              '079':'Consumer Discretionary', '081':'Consumer Staples', 
              '082':'Industrials', '083':'Industrials',
              '084':'Communication Services', '086':'Industrials',
              '087':'Consumer Discretionary', '089':'Industrials', 
              '091':'Automobiles', '092':'Automobiles', '095':'Energy',
              '096':'Energy', '099':'Consumer Staples', 
              '100':'Industrials', '101':'Information Technology', 
              '102':'Industrials', '104':'Industrials', 
              '105':'Industrials', '106':'Materials',
              '108':'Industrials', '109':'Industrials', 
              '110':'Utilities', '111':'Consumer Discretionary', 
              '112':'Consumer Staples', '114':'Industrials', 
              '116':'Industrials', '117':'Materials',
              '118':'Materials', '119':'Health Care', '122':'Utilities', 
              '123':'Automobiles', '124':'Industrials', '125':'Materials',
              '126':'Consumer Staples', '127':'Consumer Staples', 
              '128':'Health Care', '131':'Consumer Staples', 
              '132':'Consumer Staples', '134':'Consumer Staples', 
              '135':'Industrials', '136':'Industrials', '137':'Utilities', 
              '138':'Industrials', '139':'Consumer Discretionary', 
              '140':'Industrials', '141':'Utilities', '142':'Materials',
              '144':'Materials', '147':'Industrial', '148':'Materials',
              '149':'Industrials', '150':'Consumer Discretionary', 
              '152':'Automobiles', '156':'Materials', '157':'Automobiles',
              '159':'Materials', '160':'Industrials', '162':'Materials', 
              '163':'Industrials', '164':'Materials', '165':'Utilities',
              '166':'Utilities', '168':'Health Care', 
              '169':'Consumer Staples', '171':'Consumer Discretionary', 
              '172':'Industrials', '173':'Industrials', 
              '174':'Industrials', '175':'Industrials',
              '177':'Consumer Staples', '178':'Communication Services',
              '180':'Automobiles', '181':'Information Technology',
              '182':'Industrials', '184':'Industrials', 
              '185':'Automobiles', '186':'Consumer Discretionary', 
              '187':'Industrials', '188':'Automobiles', '190':'Materials',
              '191':'Industrials', '192':'Automobiles', '193':'Industrials',
              '194':'Industrials', '196':'Materials', '198':'Industrials',
              '199':'Materials', '200':'Industrials', '201':'Materials', 
              '202':'Materials', '203':'Materials', '204':'Materials', 
              '205':'Materials', '206':'Materials', '208':'Materials', 
              '209':'Industrials', '210':'Materials', '211':'Industrials',
              '212':'Industrials', '213':'Industrials', '215':'Materials',
              '216':'Materials', '217':'Materials', '218':'Industrials', 
              '219':'Industrials', '220':'Materials', '221':'Industrials',
              '222':'Industrials', '223':'Industrials', 
              '224':'Industrials', '225':'Industrials', '226':'Materials',
              '227':'Industrials', '228':'Materials', '229':'Materials', 
              '231':'Consumer Discretionary', '232':'Industrials', 
              '234':'Consumer Staples', '235':'Industrials', 
              '236':'Utilities', '237':'Utilities', 
              '238':'Industrials', '239':'Industrials', '241':'Materials', 
              '242':'Industrials', '244':'Industrials', 
              '245':'Industrials', '246':'Industrials',
              '248':'Industrials', '249':'Materials', '250':'Industrials', 
              '251':'Utilities', '252':'Materials', '254':'Industrials', 
              '256':'Industrials', '257':'Information Technology', 
              '258':'Industrials', '260':'Materials', '261':'Materials', 
              '264':'Materials', '266':'Materials', 
              '267':'Consumer Discretionary', '269':'Industrials', 
              '270':'Materials', '271':'Industrial', 
              '273':'Consumer Discretionary', 
              '276':'Information Technology', '277':'Materials', 
              '278':'Automobiles', '279':'Industrials', 
              '280':'Automobiles', '281':'Consumer Discretionary',  
              '283':'Materials', '285':'Industrials', '289':'Industrials',
              '290':'Industrials', '291':'Automobiles', 
              '292':'Industrials', '293':'Automobiles', 
              '294':'Industrials', '295':'Industrials', 
              '296':'Automobiles', '297':'Consumer Discretionary', 
              '298':'Automobiles', '299':'Materials', 
              '300':'Consumer Staples', '301':'Automobiles', 
              '303':'Automobiles', '305':'Automobiles', 
              '307':'Communication Services', '310':'Industrials', 
              '312':'Industrials', '313':'Utilities', '314':'Utilities', 
              '315':'Utilities', '318':'Industrials', '320':'Industrials', 
              '322':'Industrials', '323':'Industrials', 
              '324':'Information Technology', 
              '326':'Information Technology', 
              '327':'Industrials', '329':'Information Technology', 
              '330':'Information Technology', 
              '331':'Information Technology', 
              '332':'Information Technology', 
              '333':'Information Technology', 
              '334':'Information Technology', 
              '335':'Industrials', '336':'Industrials', 
              '337':'Industrials', '338':'Industrials', 
              '340':'Information Technology', 
              '341':'Information Technology', 
              '342':'Information Technology', 
              '343':'Information Technology', 
              '345':'Information Technology', 
              '346':'Information Technology', 
              '347':'Information Technology', 
              '348':'Information Technology', 
              '349':'Information Technology', 
              '351':'Health Care', 
              '352':'Information Technology', 
              '353':'Information Technology', 
              '355':'Information Technology', 
              '356':'Information Technology', 
              '358':'Information Technology', 
              '359':'Materials', '360':'Information Technology', 
              '361':'Industrials', '362':'Industrials', 
              '363':'Industrials', '365':'Information Technology', 
              '366':'Materials', '367':'Information Technology', 
              '368':'Information Technology', 
              '369':'Information Technology', 
              '370':'Information Technology', '372':'Industrials', 
              '373':'Utilities', '374':'Utilities', 
              '375':'Information Technology', '376':'Materials', 
              '377':'Information Technology', 
              '378':'Information Technology', 
              '379':'Information Technology', 
              '380':'Information Technology', 
              '381':'Information Technology', 
              '382':'Information Technology', 
              '383':'Materials', '384':'Industrials', 
              '385':'Information Technology', 
              '386':'Information Technology', 
              '388':'Industrials', '392':'Industrials', 
              '396':'Information Technology', 
              '398':'Information Technology', 
              '399':'Information Technology', 
              '400':'Industrials', '401':'Materials', 
              '402':'Materials', '403':'Industrials', 
              '404':'Industrials', '405':'Industrials', 
              '406':'Industrials', '407':'Industrials', 
              '408':'Industrials', '409':'Industrials', 
              '410':'Materials', '411':'Industrials', 
              '412':'Materials', '413':'Materials', '414':'Materials', 
              '415':'Industrials', '416':'Industrials', 
              '417':'Industrials', '418':'Industrials', 
              '419':'Materials', '420':'Materials', 
              '422':'Materials', '423':'Materials', 
              '424':'Health Care', '425':'Materials', 
              '426':'Consumer Staples', '427':'Materials', 
              '428':'Materials', '429':'Materials', 
              '430':'Health Care', '431':'Industrials', 
              '432':'Utilities', '433':'Health Care', 
              '434':'Information Technology', 
              '435':'Health Care', '436':'Health Care', 
              '438':'Information Technology', 
              '439':'Industrials', '440':'Industrials', 
              '441':'Industrials', 
              '442':'Consumer Discretionary', 
              '445':'Utilities', 
              '446':'Consumer Discretionary', 
              '449':'Consumer Discretionary', 
              '450':'Consumer Discretionary', 
              '451':'Industrials', 
              '452':'Consumer Staples', 
              '453':'Financials', 
              '454':'Consumer Discretionary', 
              '455':'Communication Services', 
              '460':'Industrials', 
              '462':'Consumer Discretionary', 
              '463':'Consumer Discretionary', 
              '464':'Industrials', 
              '470':'Industrials', 
              '473':'Consumer Discretionary', 
              '474':'Industrials', 
              '475':'Industrials', 
              '476':'Industrials',
              '477':'Automobiles', 
              '482':'Health Care', 
              '483':'Industrials', '492':'Industrials', 
              '493':'Materials', '494':'Materials', 
              '501':'Materials', '502':'Materials', 
              '503':'Materials', '504':'Health Care', 
              '505':'Information Technology', 
              '506':'Materials', 
              '507':'Energy', '508':'Materials', 
              '510':'Materials', '512':'Consumer Staples', 
              '514':'Health Care', '516':'Materials', 
              '518':'Materials', 
              '520':'Materials', '521':'Materials', 
              '522':'Materials', '523':'Materials', 
              '524':'Materials', '525':'Materials', 
              '526':'Materials', '527':'Materials', 
              '528':'Materials', '530':'Materials', 
              '532':'Materials', '534':'Materials', 
              '536':'Materials', '540':'Materials', 
              '544':'Materials', '546':'Materials', 
              '548':'Materials', '549':'Materials', 
              '552':'Materials', '554':'Materials', 
              '556':'Materials', '558':'Materials', 
              '560':'Materials', '562':'Materials', 
              '564':'Materials', '568':'Materials', 
              '570':'Materials', '585':'Materials', 
              '588':'Materials', '600':'Health Care', 
              '601':'Health Care', '602':'Health Care', 
              '604':'Health Care', '606':'Health Care', 
              '607':'Health Care', '623':'Health Care', 
              '700':'Information Technology', 
              '701':'Information Technology', 
              '702':'Information Technology', 
              '703':'Information Technology', 
              '704':'Information Technology', 
              '705':'Information Technology', 
              '706':'Information Technology', 
              '707':'Information Technology', 
              '708':'Information Technology', 
              '709':'Information Technology', 
              '710':'Information Technology', 
              '711':'Information Technology', 
              '712':'Information Technology', 
              '713':'Information Technology', 
              '714':'Information Technology', 
              '715':'Information Technology', 
              '716':'Information Technology', 
              '717':'Information Technology', 
              '718':'Information Technology', 
              '719':'Information Technology', 
              '720':'Information Technology', 
              '725':'Information Technology', 
              '726':'Information Technology', 
              '800':'Health Care', '850':'Health Care', 
              '901':'Information Technology', 
              '902':'Financials', '903':'Automobiles', 
              '930':'Health Care', 
              '976':'Materials', '977':'Health Care', 
              '984':'Consumer Discretionary', 
              '987':'Materials', 'D01':'Consumer Staples', 
              'D02':'Consumer Discretionary', 
              'D03':'Consumer Discretionary', 
              'D04':'Consumer Staples', 
              'D05':'Consumer Discretionary', 
              'D06':'Consumer Discretionary', 
              'D07':'Consumer Staples', 
              'D08':'Consumer Staples', 
              'D09':'Materials', 
              'D10':'Consumer Discretionary', 
              'D11':'Consumer Discretionary', 
              'D12':'Industrials', 
              'D13':'Industrials', 
              'D14':'Information Technology', 
              'D15':'Industrials', 
              'D16':'Information Technology', 
              'D17':'Consumer Discretionary', 
              'D18':'Information Technology', 
              'D19':'Consumer Discretionary', 
              'D20':'Communication Services', 
              'D21':'Consumer Discretionary', 
              'D22':'Industrials', 
              'D23':'Utilities', 
              'D24':'Health Care', 
              'D25':'Industrials', 
              'D26':'Industrials', 'D27':'Consumer Staples', 
              'D28':'Consumer Staples', 
              'D29':'Industrials', 
              'D30':'Health Care', 'D32':'Consumer Staples', 
              'D34':'Materials'}

print(len(industries))

468


In [112]:
# Create merged dataframe
merged = app_df.merge(correspondence_df, how='left', left_on='application_number', right_on='application_number')
merged.dropna(subset=['uspc_class'], inplace=True)

merged.head()

,application_number,filing_date,application_invention_type,uspc_class,uspc_subclass,correspondence_name,customer_number
0,04453098,NaN,NaN,338,254000,NaN,NaN
1,04544040,NaN,NaN,417,353000,NaN,NaN
2,04577552,NaN,NaN,060,053000,NaN,NaN
3,04578318,NaN,NaN,409,163000,NaN,NaN
4,04589018,NaN,Utility,220,271000,NaN,NaN


In [219]:
# Get rid of quotes and regroup all companies
def remove_chars_and_lower(string, chars, words):
    '''
    Function to remove quotation marks from a string, since there seems to be some issues of excess quotes
    placed into the company names
    '''
    string = ''.join(i for i in string if i not in chars)
    
    string_words = string.split()
    result_words  = [word for word in string_words if word.lower() not in words]
    result = ' '.join(result_words)
    
    return result.lower()

drop_chars = ['"', '.', '%']

drop_words = ['.com', 'llc', 'incorporated', 'inc', 'co', 'holdings', 'ho', 'corporation', 'corp', 
              'company', 'hldg', 'ent', 'ltd', 'association', 'foundation', 'club', 'fund', 'the', 'and', 'of', 
              'services', 'enterprises']

merged.dropna(subset=['correspondence_name'], inplace=True)
merged['correspondence_name'] = merged['correspondence_name'].apply(lambda x: remove_chars_and_lower(x, drop_chars, 
                                                                                                     drop_words))
merged

,application_number,filing_date,application_invention_type,uspc_class,uspc_subclass,correspondence_name,customer_number,industry
14,04667624,NaN,Utility,089,036010,edward j kelly,NaN,Industrials
16,04776670,NaN,NaN,331,179000,peter c van der sluys,NaN,Information Technology
17,04798268,NaN,Utility,260,944000,edward j kelly,NaN,Materials
23,05106194,NaN,Utility,033,304000,paul h johnson,NaN,Consumer Discretionary
35,05288968,NaN,Utility,178,695000,peter c van der sluys,NaN,Communication Services
...,...,...,...,...,...,...,...,...
16514648,17214093,2021-03-26,Re-Issue,250,NaN,foley & lardner llp,154760.0,Industrials
16514655,16603150,2021-03-19,Utility,405,079000,"carter, deluca & farrell llp",31554.0,Industrials
16514658,17221267,2021-04-02,Re-Issue,366,NaN,baker botts llp,15604.0,Materials
16514659,17223257,2021-04-06,Re-Issue,370,NaN,studebaker & brackett pc,78198.0,Information Technology


In [220]:
# Group by firm, sum the number of counts for each industry,
# and find the industry which occurs most frequently
industries_df = merged.groupby(['correspondence_name', 'industry']).count()
industries_df.to_csv('industries_df.csv')

industries_df = pd.read_csv('industries_df.csv').groupby('correspondence_name').max()
idx = industries_df.groupby(['correspondence_name'])['application_number'].transform(max) == industries_df['application_number']

industries_df = industries_df[idx]
industries_df = industries_df[['industry']]

# Fix the bad matches
complete['correspondence_name'] = summed['correspondence_name'].apply(lambda x: remove_chars_and_lower(x, drop_chars,
                                                                                                       drop_words))
complete_grouped = complete.groupby(['correspondence_name', 'Year']).sum()

complete_grouped.to_csv('complete_summed_data.csv')
complete_grouped = pd.read_csv('complete_summed_data.csv')

# Fill in all of the missing years in pandas dataframe with 0 patents
mux = pd.MultiIndex.from_product([
        complete_grouped.correspondence_name.unique(),
        range(2000, summed.Year.max() + 1)
      ], names=['correspondence_name', 'Year'])

complete_grouped = complete_grouped.set_index(['correspondence_name', 'Year']).reindex(mux).reset_index()
complete_grouped['patents'] = complete_grouped['patents'].fillna(0)

# Merge this set of industries into the cleaned summed_data
complete_grouped = complete_grouped.merge(industries_df, how='left', left_on='correspondence_name', right_index=True)
complete_grouped = complete_grouped[['correspondence_name', 'Year', 'patents', 'industry']]
complete_grouped.dropna(inplace=True)
complete_grouped.to_csv('complete_summed_data.csv')
complete_grouped

,correspondence_name,Year,patents,industry
22,"& bicknell, attn: d j brott",2000,0.0,Health Care
23,"& bicknell, attn: d j brott",2001,0.0,Health Care
24,"& bicknell, attn: d j brott",2002,0.0,Health Care
25,"& bicknell, attn: d j brott",2003,0.0,Health Care
26,"& bicknell, attn: d j brott",2004,0.0,Health Care
...,...,...,...,...
8478685,ãyvind berg,2017,0.0,Consumer Staples
8478686,ãyvind berg,2018,0.0,Consumer Staples
8478687,ãyvind berg,2019,0.0,Consumer Staples
8478688,ãyvind berg,2020,0.0,Consumer Staples


## Dirty matching

In [221]:
stanford = pd.read_csv('stanford_counts.csv')
stanford['Infringer'] = stanford['Infringer'].apply(lambda x: remove_chars_and_lower(x, drop_chars, drop_words))
stanford

,troll_count,sued_count,case_node_id,Filing_Date,asserter_id,Asserter_Category,Infringer
0,0,1,175119,3/11/16,88315,8,#1 wholesale
1,0,1,118527,2/8/00,44396,9,&quot
2,0,2,118563,8/16/01,73819,8,&quot
3,0,3,118615,1/10/03,97673,8,&quot
4,0,4,118629,9/5/03,58898,5,&quot
...,...,...,...,...,...,...,...
160881,19,32,230035,1/28/21,228761,1,zyxel communications
160882,1,1,215740,5/29/19,211701,1,zyxel networks
160883,2,2,217338,10/31/19,205264,1,zyxel networks
160884,3,3,221022,2/26/20,203838,1,zyxel networks


In [226]:
lit_df = complete_grouped.merge(stanford, how='left', left_on='correspondence_name', right_on='Infringer')
print(len(lit_df.dropna()['correspondence_name'].unique()))

2097


In [223]:
public = pd.read_csv('company_size.csv')
public['Company'] = public['Company'].apply(lambda x: x.lower())
public['Company'] = public['Company'].apply(lambda x: remove_chars_and_lower(x, drop_chars, drop_words))
public

,Year,Company,Currency,Total Assets (millions),Employees (thousands),Company Status,Market Cap (millions),State Incorporated,S&P industry code,State
0,1999.0,aar,USD,740.998,2.900,A,372.7519,DE,110.0,IL
1,2000.0,aar,USD,701.854,2.500,A,377.1180,DE,110.0,IL
2,2001.0,aar,USD,710.199,2.200,A,364.5928,DE,110.0,IL
3,2002.0,aar,USD,686.621,2.100,A,143.3295,DE,110.0,IL
4,2003.0,aar,USD,709.292,2.300,A,308.9071,DE,110.0,IL
...,...,...,...,...,...,...,...,...,...,...
274262,2020.0,karooooo,USD,192.079,3.122,A,NaN,NaN,NaN,NaN
274263,2019.0,iveco group n v,USD,17847.000,32.525,A,NaN,NaN,NaN,NaN
274264,2020.0,iveco group n v,USD,18841.000,32.632,A,NaN,NaN,NaN,NaN
274265,2019.0,daimler truck holding ag,USD,62131.888,99.021,A,NaN,NaN,NaN,NaN


In [227]:
full_lit_df = lit_df.merge(public, how='left', left_on='correspondence_name', right_on='Company')
print(len(full_lit_df.dropna()['correspondence_name'].unique()))

319
